# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_weather_data=pd.read_csv("../output_data/clean_Weather_data.csv")
clean_weather_data


,City,Latitude,Longitude,Country,Temperate (°C),Min Temp(°C),Max Temp(°C),Humidity,Cloudiness,windspeed
0,Ushuaia,-54.80,-68.30,AR,0.00,0.00,0.00,90,40,4.10
1,Rikitea,-23.12,-134.97,PF,22.66,22.66,22.66,84,100,7.15
2,Mataura,-46.19,168.86,NZ,6.49,6.49,6.49,90,93,1.49
3,Srīvardhan,18.03,73.02,IN,25.67,25.67,25.67,91,100,1.81
4,Whitehorse,60.72,-135.05,CA,12.46,9.44,14.00,54,75,4.10
...,...,...,...,...,...,...,...,...,...,...
572,Hukuntsi,-24.00,21.75,BW,2.87,2.87,2.87,94,0,4.17
573,Kita,13.03,-9.49,ML,21.96,21.96,21.96,90,100,1.52
574,Khāsh,28.22,61.22,IR,33.82,33.82,33.82,23,0,2.25
575,Santa Cruz de la Sierra,-17.80,-63.17,BO,21.00,21.00,21.00,88,75,4.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
Coordinates = clean_weather_data[["Latitude", "Longitude"]]
Humidity=clean_weather_data["Humidity"]


In [6]:
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig =gmaps.figure(layout=figure_layout)

heat_layer=gmaps.heatmap_layer(Coordinates, weights=Humidity, dissipating=False, max_intensity=20, point_radius=2)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [7]:
# markers = gmaps.marker_layer(Coordinates)
# # Add the layer to the map
# fig.add_layer(markers)
# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_hol = clean_weather_data.loc[clean_weather_data["Max Temp(°C)"] <35 ]
ideal_hol =ideal_hol.loc[ideal_hol["Max Temp(°C)"] >23]
ideal_hol = ideal_hol.loc[ideal_hol["Cloudiness"]==0]
ideal_hol = ideal_hol.loc[ideal_hol["windspeed"] <10]
ideal_hol

,City,Latitude,Longitude,Country,Temperate (°C),Min Temp(°C),Max Temp(°C),Humidity,Cloudiness,windspeed
143,Marsá Maţrūḩ,31.35,27.25,EG,24.00,24.00,24.00,83,0,4.60
190,Sarkand,45.41,79.92,KZ,25.17,25.17,25.17,34,0,3.73
243,Sergeevka,53.88,67.42,KZ,27.30,27.30,27.30,56,0,0.66
248,Hurghada,27.26,33.81,EG,32.00,32.00,32.00,46,0,3.10
268,Suruç,36.98,38.42,TR,29.00,29.00,29.00,30,0,7.20
292,Adrar,20.50,-10.07,MR,31.47,31.47,31.47,30,0,4.78
326,Cap Malheureux,-19.98,57.61,MU,24.44,24.44,24.44,69,0,3.13
328,Khorramabad,33.49,48.36,IR,32.00,32.00,32.00,17,0,2.10
331,Ubari,26.59,12.78,LY,24.40,24.40,24.40,36,0,4.55
347,Atasū,48.68,71.64,KZ,26.98,26.98,26.98,31,0,7.35


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df= ideal_hol
hotel_df["Hotel Name"]=""
hotel_df

,City,Latitude,Longitude,Country,Temperate (°C),Min Temp(°C),Max Temp(°C),Humidity,Cloudiness,windspeed,Hotel Name
143,Marsá Maţrūḩ,31.35,27.25,EG,24.00,24.00,24.00,83,0,4.60,
190,Sarkand,45.41,79.92,KZ,25.17,25.17,25.17,34,0,3.73,
243,Sergeevka,53.88,67.42,KZ,27.30,27.30,27.30,56,0,0.66,
248,Hurghada,27.26,33.81,EG,32.00,32.00,32.00,46,0,3.10,
268,Suruç,36.98,38.42,TR,29.00,29.00,29.00,30,0,7.20,
292,Adrar,20.50,-10.07,MR,31.47,31.47,31.47,30,0,4.78,
326,Cap Malheureux,-19.98,57.61,MU,24.44,24.44,24.44,69,0,3.13,
328,Khorramabad,33.49,48.36,IR,32.00,32.00,32.00,17,0,2.10,
331,Ubari,26.59,12.78,LY,24.40,24.40,24.40,36,0,4.55,
347,Atasū,48.68,71.64,KZ,26.98,26.98,26.98,31,0,7.35,


In [36]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keywords": "hotel",
    "key": g_key, 
    "language": "en"

}

# Use the lat/lng we recovered to identify airports


# get lat, lng from df
lat = row["Latitude"]
lng = row["Longitude"]

# change location each iteration while leaving original params in place
params["location"] = f"{lat},{lng}"

# Use the search term: "Hotels" and our lat/lng
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

 # make request and print url
    
hotel_nearby = requests.get(base_url, params=params).json()
hotel_nearby

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 28.2171113, 'lng': 61.21680399999999},
    'viewport': {'northeast': {'lat': 28.2184483302915,
      'lng': 61.2181391302915},
     'southwest': {'lat': 28.2157503697085, 'lng': 61.21544116970849}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '6a6d1e441f61a2b518b75c4b6af477355a5ff50b',
   'name': 'دفتر پیشخوان دولت جمال زهی (خالد)',
   'place_id': 'ChIJG3FiTLMF6D4RqSg5vxNaZjo',
   'plus_code': {'compound_code': '6688+RP Khash, Sistan and Baluchestan Province, Iran',
    'global_code': '7JW36688+RP'},
   'reference': 'ChIJG3FiTLMF6D4RqSg5vxNaZjo',
   'scope': 'GOOGLE',
   'types': ['lodging', 'point_of_interest', 'establishment'],
   'vicinity': 'Khash'},
  {'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 28.2170927, 'lng': 61.2148021},
    'viewport': {'northeast': {'lat': 28.2184134302915,
      'lng': 61.21618

In [50]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "keywords": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():

     # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
        
    #defind city for each for - for print statement
    city=row["City"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotels" and our lat/lng
        

 # make request and print url
    try:
        hotel_nearby = requests.get(base_url, params=params).json()
        hotel_name=hotel_nearby["results"][0]["name"]
        hotel_df.loc[index, 'Hotel Name']=hotel_name
        
        print(f"Retrieving hotel for {city}" )
          
    except:
        
        print(f"No hotel found for {city}")

Retrieving hotel for Marsá Maţrūḩ
Retrieving hotel for Sarkand
No hotel found for Sergeevka
Retrieving hotel for Hurghada
Retrieving hotel for Suruç
No hotel found for Adrar
Retrieving hotel for Cap Malheureux
Retrieving hotel for Khorramabad
Retrieving hotel for Ubari
Retrieving hotel for Atasū
No hotel found for Umm Kaddadah
Retrieving hotel for Buraidah
Retrieving hotel for Bamiantong
Retrieving hotel for Şabrātah
Retrieving hotel for Derzhavīnsk
Retrieving hotel for Port Hedland
Retrieving hotel for Cabra
Retrieving hotel for Palaia Fokaia
Retrieving hotel for Suez
No hotel found for Berdigestyakh
Retrieving hotel for Sakakah
Retrieving hotel for Khāsh


In [47]:
hotel_df

,City,Latitude,Longitude,Country,Temperate (°C),Min Temp(°C),Max Temp(°C),Humidity,Cloudiness,windspeed,Hotel Name
143,Marsá Maţrūḩ,31.35,27.25,EG,24.00,24.00,24.00,83,0,4.60,Beau Site
190,Sarkand,45.41,79.92,KZ,25.17,25.17,25.17,34,0,3.73,Гостиница Сарканд
243,Sergeevka,53.88,67.42,KZ,27.30,27.30,27.30,56,0,0.66,دفتر پیشخوان دولت جمال زهی (خالد)
248,Hurghada,27.26,33.81,EG,32.00,32.00,32.00,46,0,3.10,Sea Star Beau Rivage Hotel
268,Suruç,36.98,38.42,TR,29.00,29.00,29.00,30,0,7.20,Kumbet Otel
292,Adrar,20.50,-10.07,MR,31.47,31.47,31.47,30,0,4.78,دفتر پیشخوان دولت جمال زهی (خالد)
326,Cap Malheureux,-19.98,57.61,MU,24.44,24.44,24.44,69,0,3.13,Pereybere Hotel & Spa
328,Khorramabad,33.49,48.36,IR,32.00,32.00,32.00,17,0,2.10,Azadi Hotel
331,Ubari,26.59,12.78,LY,24.40,24.40,24.40,36,0,4.55,فندق أوباري
347,Atasū,48.68,71.64,KZ,26.98,26.98,26.98,31,0,7.35,Dostyk


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [59]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…